# Busca usando Embeddings

In [1]:
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import torch
import numpy as np
import pandas as pd

model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

fine_tuned_embeddings = np.load('../embeddings/fine_tuned_embeddings.npy')
autoencoder = torch.load('../eembeddings/autoencoder_model.pt') 
autoencoder.eval()  

sephora_data = pd.read_csv('../data/sephora_reviews.csv')

def semantic_search(query, top_n=10, threshold=0.2):
    query_embedding = model.encode([query])

    query_embedding_tensor = torch.tensor(query_embedding).float()
    query_embedding_reduced = autoencoder.encoder(query_embedding_tensor) 
    query_embedding_reduced = query_embedding_reduced.detach().numpy()

    similarities = cosine_similarity(query_embedding_reduced, fine_tuned_embeddings).flatten()

    sephora_data['Similarity'] = similarities
    filtered_results = sephora_data[sephora_data['Similarity'] > threshold]
    sorted_results = filtered_results.sort_values(by='Similarity', ascending=False)

    sorted_results = sorted_results.drop_duplicates(subset='Nome')

    final_results = sorted_results[['Nome', 'Similarity']].head(min(top_n, len(sorted_results)))
    result_count = len(final_results)

    print(f"Quantidade de resultados: {result_count}")
    return final_results



/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning:

`clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884

<ipython-input-24-720d80502f71>:12: FutureWarning:

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are ex

## Queries

In [25]:
# Consultas de exemplo para testar o sistema de busca
query_10_results = "acne"
query_less_than_10_results = "seco"
query_non_obvious = "manchas na pele"

## Resultados

In [26]:
# Executar os testes
results_10 = semantic_search(query_10_results)
results_few = semantic_search(query_less_than_10_results, threshold=0.9)
results_non_obvious = semantic_search(query_non_obvious)

Quantidade de resultados: 10
Quantidade de resultados: 2
Quantidade de resultados: 10


In [28]:
print("\nResultados para a consulta (10 resultados):")
print(results_10)


Resultados para a consulta (10 resultados):
                                                  Nome  Similarity
1    GEL DE LIMPEZA FACIAL ANTI-IMPERFEIÇÕES SEPHOR...    0.942546
881   TÔNICO ROSE INC SKIN RESOLUTION CLARIFYING TONER    0.937030
548  ÓLEO FACIAL HIDRATANTE DRUNK ELEPHANT VIRGIN M...    0.935236
144  CREME COM CERAMIDAS PARA OS OLHOS DRUNK ELEPHA...    0.933941
681                    MOUSSE GEL DE LIMPEZA GEL ÉCLAT    0.933483
226      DOUBLE SÉRUM AVANÇADO GUERLAIN ABEILLE ROYALE    0.929882
717     ESPUMA DE LIMPEZA SHISEIDO DEEP CLEANSING FOAM    0.928371
870  CREME PARA OLHOS ROSE INC EYE REVIVAL BRIGHTEN...    0.927898
332  ÓLEO REJUVENESCEDOR AVANÇADO GUERLAIN ABEILLE ...    0.927219
764  FUTURE SOLUTION LX EXTRA RICH CLEANSING FOAM (...    0.927122


In [29]:
print("\nResultados para a consulta (menos de 10 resultados):")
print(results_few)


Resultados para a consulta (menos de 10 resultados):
                                                  Nome  Similarity
146  CREME DE LIMPEZA DERMALOGICA MICELLAR PREBIOTI...    0.911386
811              CONCENTRADO HIDRATANTE MOISTURE SURGE    0.901117


In [30]:
print("\nResultados para a consulta não óbvia:")
print(results_non_obvious)


Resultados para a consulta não óbvia:
                                                  Nome  Similarity
855                 BALM DEMAQUILANTE TAKE THE DAY OFF    0.967102
77   ÓLEO DE LIMPEZA GUERLAIN ANTI-POLUIÇÃO ABEILLE...    0.966397
451               ESPUMA DE LIMPEZA CAUDALIE VINOCLEAN    0.964809
432   HIDRATANTE EM GEL MOISTURE SURGE ™ 100H CLINIQUE    0.964466
721     ESPUMA DE LIMPEZA SHISEIDO DEEP CLEANSING FOAM    0.964270
537  CREME HIDRATANTE FACIAL DRUNK ELEPHANT PROTINI...    0.964127
314  ESPUMA DE LIMPEZA CLARINS TOTAL RENEWING FOAMI...    0.962758
867  PEELING LÍQUIDO ESFOLIANTE DERMALOGICA PEELFOL...    0.962387
429  GEL CORRETIVO PARA VERMELHIDÃO CALMANTE CLARIN...    0.962225
812              SÉRUM REPARAÇÃO DIÁRIA ABEILLE ROYALE    0.961549
